# Cognitive Architectures

How does your LLM system think? There are a few different strategies to improve reasoning ability of LLMs. Let's cover them here.

In [1]:
from operator import itemgetter

from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.runnables.openai_functions import OpenAIFunctionsRouter

from permchain.connection_inmemory import InMemoryPubSubConnection
from permchain.pubsub import PubSub
from permchain.topic import Topic

## LLM Call

Most basic - just ask the LLM.

In [2]:
message = "write a two line poem about the college that brett farves replacement in GB went to"

In [3]:
model = ChatOpenAI(temperature=0)

In [4]:
model.invoke(message)

AIMessage(content="From Southern Miss to Green Bay's fame,\nA golden eagle soared, Rodgers became.", additional_kwargs={}, example=False)

## Chain of Thought

The LLM we are using has been trained with Chain-of-Thought. For older LLMs, you had to explicitly ask the LLM to think this way.

In [5]:
simple_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "let's think step by step to get the CORRECT answer"),
])

In [6]:
simple_chain = simple_prompt | model | StrOutputParser()

In [7]:
simple_chain.invoke({"input": message})

'From Southern Miss he rose,\nTo lead the Packers, his talent shows.'

## Revise

Create a chain that revises an initial prediction

In [8]:
revise_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given a user question and initial attempt, revise the attempt be (1) more factually accurate, (2) a better poem"),
    ("human", "{input}"),
    ("system", "here's your initial attempt:"),
    ("ai", "{attempt}"),
])

In [9]:
revise_chain = {
    "input": lambda x: x["input"],
    "attempt": simple_chain | StrOutputParser()
} | revise_prompt | model

In [10]:
revise_chain.invoke({"input": message})

AIMessage(content="In Green Bay's realm, a legend grew,\nFrom Southern Miss, the chosen few.", additional_kwargs={}, example=False)

You can also do this with permchain

In [11]:
reviser_inbox = Topic("reviser_inbox")
draft_chain = (
    # Listed in inputs
    Topic.IN.subscribe()
    | {"attempt": simple_chain | StrOutputParser(), "input": Topic.IN.current() | itemgetter("input")}
    # The draft always goes to the editors inbox
    | reviser_inbox.publish()
)

reviser_chain = (
    # Listen for events in the reviser's inbox
    reviser_inbox.subscribe()
    | {"revision": revise_chain | StrOutputParser()}
    # Publish to the editors inbox
    | Topic.OUT.publish()
)

In [12]:
revise_pub_sub = PubSub(
    processes=(draft_chain, reviser_chain),
    connection=InMemoryPubSubConnection(),
)

In [13]:
revise_pub_sub.invoke({"input": message})

[{'revision': "In Green Bay's realm, a legend grew,\nFrom Southern Miss, the chosen few."}]

## Critique and Revise

Let's separate out the critique and revise steps explicitly


In [15]:
critique_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given a user question and initial attempt, critique whether the attempt is factually accurate and a valid two line poem"),
    ("human", "{input}"),
    ("system", "here's your initial attempt:"),
    ("ai", "{attempt}"),
])
revise_critique_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given a user question and initial attempt, first critique the attempt, and then revise it based on the critique"),
    ("human", "{input}"),
    ("system", "here's your initial attempt:"),
    ("ai", "{attempt}"),
    ("system", "here's your critique:"),
    ("ai", "{critique}"),
])

In [16]:
reviser_inbox = Topic("reviser_inbox")
critique_inbox = Topic("critique_inbox")
draft_chain = (
    # Listed in inputs
    Topic.IN.subscribe()
    | {"attempt": simple_chain | StrOutputParser(), "input": Topic.IN.current() | itemgetter("input")}
    # The draft always goes to the editors inbox
    | critique_inbox.publish()
)

critique_chain = (
    # Listen for events in the reviser's inbox
    critique_inbox.subscribe()
    | {
        "critique": critique_prompt | ChatOpenAI() | StrOutputParser(),
        "input": Topic.IN.current() | itemgetter("input"),
        "attempt": itemgetter("attempt")
    }
    # Publish to the editors inbox
    | reviser_inbox.publish()
)

reviser_chain = (
    # Listen for events in the reviser's inbox
    reviser_inbox.subscribe()
    | {"revision": revise_chain | StrOutputParser()}
    # Publish to the editors inbox
    | Topic.OUT.publish()
)

In [18]:
critique_revise_pub_sub = PubSub(
    processes=(draft_chain, reviser_chain, critique_chain),
    connection=InMemoryPubSubConnection(),
)

In [19]:
critique_revise_pub_sub.invoke({"input": message})

[{'revision': "In Green Bay's realm, a legend grew,\nFrom Southern Miss, the chosen few."}]

## Multiple Critique and Revise

You can also run the critique/revision step multiple times

In [20]:
editor_llm = ChatOpenAI(model="gpt-4")
functions = [
    {
        "name": "revise",
        "description": "Sends the draft for revision",
        "parameters": {
            "type": "object",
            "properties": {
                "notes": {
                    "type": "string",
                    "description": "The editor's notes to guide the revision.",
                },
            },
        },
    },
    {
        "name": "accept",
        "description": "Accepts the draft",
        "parameters": {
            "type": "object",
            "properties": {"ready": {"const": True}},
        },
    },
]
editor = critique_prompt | editor_llm.bind(functions=functions)

In [21]:
# create topics
editor_inbox = Topic("editor_inbox")
reviser_inbox = Topic("reviser_inbox")

draft_chain = (
    # Listed in inputs
    Topic.IN.subscribe()
    | {"attempt": simple_chain | StrOutputParser(), "input": Topic.IN.current() | itemgetter("input")}
    # The draft always goes to the editors inbox
    | editor_inbox.publish()
)

editor_chain = (
    # Listen for events in the editors inbox
    editor_inbox.subscribe()
    | editor
    # Depending on the output, different things should happen
    | OpenAIFunctionsRouter(
        {
            # If revise is chosen, we send a push to the revisor's inbox
            "revise": (
                {
                    "critique": itemgetter("notes"),
                    "attempt": editor_inbox.current() | itemgetter("attempt"),
                    "input": Topic.IN.current() | itemgetter("input"),
                }
                | reviser_inbox.publish()
            ),
            # If accepted, then we return
            "accept": editor_inbox.current() | Topic.OUT.publish(),
        },
    )
)

reviser_chain = (
    # Listen for events in the reviser's inbox
    reviser_inbox.subscribe()
    | {"attempt": revise_chain | StrOutputParser(), "input": Topic.IN.current() | itemgetter("input")}
    # Publish to the editors inbox
    | editor_inbox.publish()
)

revise_loop = PubSub(
    processes=(draft_chain, editor_chain, reviser_chain),
    connection=InMemoryPubSubConnection(),
)

In [22]:
revise_loop.invoke({"input": message})

[{'attempt': 'From Southern Miss he rose,\nTo lead the Packers, his talent shows.',
  'input': 'write a two line poem about the college that brett farves replacement in GB went to'}]

## Plan and Execute

In [23]:
from __future__ import annotations

import re
from abc import abstractmethod
from typing import List

from langchain.schema import BaseOutputParser


class ListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a list."""

    @property
    def _type(self) -> str:
        return "list"

    @abstractmethod
    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""


class CommaSeparatedListOutputParser(ListOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    @property
    def lc_serializable(self) -> bool:
        return True

    def get_format_instructions(self) -> str:
        return (
            "Your response should be a list of comma separated values, "
            "eg: `foo, bar, baz`"
        )

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")


class NumberedListOutputParser(ListOutputParser):
    """Parse a numbered list."""

    def get_format_instructions(self) -> str:
        return (
            "Your response should be a numbered list with each item on a new line. "
            "For example: \n\n1. foo\n\n2. bar\n\n3. baz"
        )

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        pattern = r"\d+\.\s([^\n]+)"

        # Extract the text of each item
        matches = re.findall(pattern, text)
        return matches

In [25]:
output_parser = NumberedListOutputParser()
plan_prompt = ChatPromptTemplate.from_messages([
    ("system", f"Make a plan to accomplish the user objective. {output_parser.get_format_instructions()}"),
    ("human", "{input}")
])
plan_chain = plan_prompt | ChatOpenAI() | output_parser

In [26]:
plan_chain.invoke({"input": message})

["Research the college that Brett Favre's replacement in Green Bay went to. ",
 'Write a two-line poem that captures the essence or significance of that college in relation to the replacement quarterback.']